In [1]:
import streamlit as st
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = pd.read_csv("C:\\Users\\Bill_\\Desktop\\Projetos\\EBAC\\DataScience\\Modulo 16 - Pandas e Projeto\\previsao_de_renda.csv") # Carregando o quadro de dados para previsão de rendas
df = df.drop('sexo', axis = 1) # Removendo colunas de não interesse (devido a LGPD)
df = df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1) # Colunas que não vão ajudar em nada nas análises
#
filtro_nulo = df['tempo_emprego'].isnull() # Salvando os valores nulos como True/False
df_nulo_renda = df[filtro_nulo][['tempo_emprego', 'tipo_renda','renda']] # Pegando os valores True(nulos) junto com as demais colunas
#
def gerar_intervalos_e_rotulos(margem, limite):
    
    ''' Aqui é um verificador para que não sobre a margem na hora de adicionar o ultimo elemento,
    então se fosse sobrar margem, aqui aumenta o limite para a divisao de limite e margem de zero
    independente do valor que for passado para eles.'''
    if limite % margem != 0: # Verifica resto da divisão
        limite_c_margem = (limite - (limite % margem)) + margem # Altera para o próximo valor divisivel por margem
    else:
        limite_c_margem = limite # Mantem o valor caso a divisão seja inteira

    i = 0 
    intervalos = []
    rotulos = []

    while i <= limite_c_margem:
        intervalos.append(i) # Adiciona os valores de intervalo de acordo com a margem
        ''' Esse if serve para verificar o ultimo elemento do rotulo e colocar o simbolo de 
        infinito para cima que é o "+" caso caia no else'''
        if i < limite_c_margem:
            rotulos.append(f'{i}-{i + margem}') # Adiciona os rotulos, exemplo 500-1000 ou 300-600
        else:
            rotulos.append(f'{i}+') # Adiciona o ultimo elemento do rotulo e o simbolo +, define sem limite superior
        i = i + margem
    intervalos.append(float('inf')) # Adiciona por ultimo 'inf' que é infinito, sem limite superior
    
    return intervalos, rotulos # Retorna as listas
#
df_clone = df.copy() # Gerando um DF novo pra poder mexer sem alterar o original
intervalos, rotulos = gerar_intervalos_e_rotulos(1000, 9000) # Passando os valores pra função
'''Criando a coluna renda_intervalo com intervalos dos valores de renda, esses intervalos
vão ser divididos conforme o valor de intervalos e cada pedaço criado vai ser preenchido
pelo valor de rotulos, o right serve para excluir o limite superior, então a margem vai incluir
o limite inferior.'''
df_clone['renda_intervalo'] = pd.cut(df_clone['renda'], bins=intervalos, labels=rotulos, right=False)

# Crie uma tabela de informações cruzadas usando pd.crosstab(), margins=True inclui os totais
info_cross = pd.crosstab(
    index=df_clone['posse_de_veiculo'],
    columns=df_clone['renda_intervalo'], 
    margins=True)
info_cross